## Задание 2.1 Обучите нейронную сеть решать шифр Цезаря.

Импорт библиотек.

In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import time

import random

In [ ]:
import os
import re
import sys
from google.colab import drive

In [ ]:
import copy

In [ ]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Алгоритм шифра Цезаря

In [ ]:
def Сaesar(string, num):
    output = ''
    for c in string:
        if c.isalpha():
            new_num = ord(c) + num
            if new_num > ord('z'):
                new_num -= 26
            output += chr(new_num)
        else:
            output += c
    return output

In [ ]:
Сaesar('trew', 2)

'vtgy'

Исходные данные

In [ ]:
df = pd.read_csv('/content/drive/My Drive/DLL/simpsons/data.csv').iloc[:,[-2]]
df = df.dropna(subset=['normalized_text'])

# установим произвольный сдвиг в составе шифра Цезаря до 10 символов
# шифр индивидуален для каждой записи
df['shift']   = [random.randint(1, 10) for i in range(len(df))]

# исходная реплика
df['text_in'] = [' '.join(re.findall('[\w]+', i)) for i in df['normalized_text']]

# зашифрованная реплика
df['text_out']= df.loc[:, ['text_in', 'shift']].apply(
                        lambda row: Сaesar(row['text_in'], row['shift']), axis=1)

df = df.iloc[:,1:]
df.head()

,shift,text_in,text_out
0,8,maggie look whats that,uiooqm twws epiba bpib
1,8,lee mur lee mur,tmm ucz tmm ucz
2,2,zee boo zee boo,bgg dqq bgg dqq
3,7,im trying to teach maggie that nature doesnt e...,pt ayfpun av alhjo thnnpl aoha uhabyl kvlzua l...
4,3,its like an ox only it has a hump and a dewlap...,lwv olnh dq ra rqob lw kdv d kxps dqg d ghzods...


Ограничим обучение первыми 50 символами каждой из реплик

In [ ]:
MAX_LEN       = 50
col_name = ''.join(['text_in_', str(MAX_LEN), '_symb'])
df[col_name] = [df.loc[i,'text_in'][:MAX_LEN] for i in df.index]

In [ ]:
df[col_name].value_counts().head(10)

no      52
dad     37
yes     30
what    22
bart    20
hey     19
yay     18
yeah    17
okay    17
mom     16
Name: text_in_50_symb, dtype: int64

По выборке чаще всего повторяются выражения состоящие из 3- 4 символов,  эта выборка реплик нерелевантна для задачи по генерации текста на более 20 символов. Исключим короткие реплики из наблюдения.

In [ ]:
df['text_in_len'] =[len(df.loc[i,'text_in']) for i in df.index]
df = df[df.text_in_len > 10]
df.head()

,shift,text_in,text_out,text_in_50_symb,text_in_len
0,8,maggie look whats that,uiooqm twws epiba bpib,maggie look whats that,22
1,8,lee mur lee mur,tmm ucz tmm ucz,lee mur lee mur,15
2,2,zee boo zee boo,bgg dqq bgg dqq,zee boo zee boo,15
3,7,im trying to teach maggie that nature doesnt e...,pt ayfpun av alhjo thnnpl aoha uhabyl kvlzua l...,im trying to teach maggie that nature doesnt e...,122
4,3,its like an ox only it has a hump and a dewlap...,lwv olnh dq ra rqob lw kdv d kxps dqg d ghzods...,its like an ox only it has a hump and a dewlap...,80


Импорт to Torch

In [ ]:
train, test = train_test_split(df, test_size=0.2)
train_text = [[c for c in ph] for ph in train['text_out'] if type(ph) is str]
train_label= [[c for c in ph] for ph in train['text_in'] if type(ph) is str]
test_text   = [[c for c in ph] for ph in test['text_out'] if type(ph) is str]
test_label  = [[c for c in ph] for ph in test['text_in'] if type(ph) is str]

In [ ]:
ABC           = ['none'] + [w for w in set('abcdefghijklmnopqrstuvwxyz ')]
CHAR_TO_INDEX = {w: i for i, w in enumerate(ABC)}
INDEX_TO_CHAR = {i: w for i, w in enumerate(ABC)}

def convert_to_torch(text):
    output = torch.zeros((len(text), MAX_LEN), dtype=int)
    for i in range(len(text)):
        for j, w in enumerate(text[i]):
            if j >= MAX_LEN:
                break
            output[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

    return output

X_train= convert_to_torch(train_text)
Y_train= convert_to_torch(train_label)
X_test = convert_to_torch(test_text)
Y_test = convert_to_torch(test_label)

Базовая архтектура сети

In [ ]:
class RNN_Network(torch.nn.Module):
    def __init__(self):
        super(RNN_Network, self).__init__()
        self.embeddings = torch.nn.Embedding(len(ABC), 28)
        self.rnn = torch.nn.RNN(28, 256, batch_first=True)
        self.linear = torch.nn.Linear(256, 28)

    def forward(self, sentences, state=None):
        embds = self.embeddings(sentences)
        out, new_state = self.rnn(embds, state)
        result = self.linear(out)
        return result, new_state

In [ ]:
model = RNN_Network().to(dev)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
epochs    = 40
loss_best = 10**10

Обучение сети

In [ ]:
for epoch in range(epochs):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    test_loss = 0
    test_passed = 0

    for i in range(int(len(X_train) / 100)):
        X_batch = X_train[i * 100:(i + 1) * 100].to(dev)
        Y_batch = Y_train[i * 100:(i + 1) * 100].flatten().to(dev)
        model.train()
        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(ABC))
        loss = criterion(answers, Y_batch).to(dev)

        if loss < loss_best:
            model_best = copy.copy(model)#.to(dev)
            loss_best = loss#.to(dev)

        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_passed += 1

    with torch.no_grad():
        model.eval()
        answers, _ = model.forward(X_test.to(dev))
        answers = answers.view(-1, len(ABC))
        loss = criterion(answers, Y_test.flatten().to(dev))
        test_loss += loss.item()
        test_passed += 1

    if epoch%1 == 0:
        print(f"Epoch {epoch}. Time: {time.time() - start:.3f}, Train loss: {train_loss / train_passed:.3f}, Test loss: {test_loss / test_passed:.6f}")

Epoch 0. Time: 1.024, Train loss: 1.976, Test loss: 1.609095
Epoch 1. Time: 0.852, Train loss: 1.450, Test loss: 1.321104
Epoch 2. Time: 0.876, Train loss: 1.243, Test loss: 1.187204
Epoch 3. Time: 0.847, Train loss: 1.153, Test loss: 1.129084
Epoch 4. Time: 0.839, Train loss: 1.108, Test loss: 1.096524
Epoch 5. Time: 0.844, Train loss: 1.079, Test loss: 1.072614
Epoch 6. Time: 0.844, Train loss: 1.056, Test loss: 1.051958
Epoch 7. Time: 0.844, Train loss: 1.035, Test loss: 1.031785
Epoch 8. Time: 0.841, Train loss: 1.011, Test loss: 1.002696
Epoch 9. Time: 0.842, Train loss: 0.972, Test loss: 0.953110
Epoch 10. Time: 0.876, Train loss: 0.914, Test loss: 0.888802
Epoch 11. Time: 0.842, Train loss: 0.853, Test loss: 0.851472
Epoch 12. Time: 0.841, Train loss: 0.788, Test loss: 0.778426
Epoch 13. Time: 0.839, Train loss: 0.748, Test loss: 0.736192
Epoch 14. Time: 0.847, Train loss: 0.702, Test loss: 0.698213
Epoch 15. Time: 0.846, Train loss: 0.658, Test loss: 0.659878
Epoch 16. Time: 0.

In [ ]:
loss_best

tensor(0.1487, device='cuda:0', grad_fn=<NllLossBackward>)

Расшифровка текста.

In [ ]:
ZZZ = convert_to_torch([[c for c in ph] for ph in df['text_out']
                                        if type(ph) is str])


df['text_predict']= [''.join([INDEX_TO_CHAR[i.item()] for i
                    in model_best(ZZZ.to(dev))[0][line].argmax(dim=1).detach()])
                    for line in range(df.shape[0])]

In [ ]:
df.iloc[170:185,[3,5]]

,text_in_50_symb,text_predict
203,dad can i have some money to buy bart a birthd...,iad can i have some money to buy bart a birthd...
204,dad this is a hundred and ten dollars,dad this is a hundred and ten dollarsnonenonen...
205,quit it dad,wamt it dadnonenonenonenonenonenonenonenonenon...
206,i dreamed i was married to corey and we lived ...,i dreamed i was married to corey and we lived ...
207,is it sherri,is it sherrinonenonenonenonenonenonenonenoneno...
208,how do you mean dad,iow do you mean dadnonenonenonenonenonenonenon...
209,how does that make him a hero,imw does that make him a herononenonenonenonen...
212,bart simpson the thought of a boy trapped in a...,iart simpson the thought of a boy trapped in a...
213,the police will catch you sooner or later,whe police will catch you sooner or laternonen...
214,maybe not but i bet youre stupid enough to hav...,iayde oot but i bet youre stupid enough to hav...
